In [ ]:
#

In [1]:
# dependencies
import pandas as pd 
import requests
import json
import gmaps
import datetime
import os
import glob

In [79]:
#map config
from config import gkey 
base_url = "https://maps.googleapis.com/maps/api/geocode/json?" 
params = {"key": gkey}  

In [49]:
# this function returns raw tweet data by given month
def raw_data_by_month(month):
    
    #curr_path = os.getcwd() 
    #print(curr_path)
    
    #os.chdir(r'../Resources/twitterapi/')  

    files = [i for i in glob.glob('{}*.{}'.format(month,'csv'))] 

    raw_df = pd.concat([pd.read_csv(f) for f in files ])  
    
    return raw_df

In [135]:
# this function filters&groups dataframe
def filter_data(raw_df):
    
    filtered_df = raw_df.loc[:, ['id', 'user_location']] 
    filtered_df = filtered_df[filtered_df['user_location'].notna()]
    
    # filter nc state
    location_suffixes = (', nc', ',nc', 'n.c.', 'n.c', ', north carolina')

    filtered_df = filtered_df[filtered_df['user_location'].str.lower().str.endswith(location_suffixes) ]  
 
    #group by data for google api request #iterrow
    filtered_df = filtered_df.groupby('user_location')['id'].count().to_frame().reset_index()
    #ex: df.groupby('product')['value'].sum().to_frame().reset_index()
    
    #reset index  
    #filtered_df.reset_index() 
    #del filtered_df['index'] 
    
    return filtered_df

In [165]:
def append_geocode_to_df(grouped_df):

    for index, row in filtered_df.iterrows():
        place = row['user_location'] 
 
        params['address'] = f"{place}" 
    
        resp_data = requests.get(base_url, params = params).json() 
        #print(resp_data)

        try: 
            #if resp_data['results'][0] is None:
            #    print('None')
            grouped_df.loc[index, 'Latitude']  = resp_data['results'][0]['geometry']['location']['lat']
            grouped_df.loc[index, 'Longitude'] = resp_data['results'][0]['geometry']['location']['lng'] 
        except (KeyError, IndexError):
            print("missing result. skipping")
            pass

    grouped_df = grouped_df.dropna()
    return grouped_df

In [166]:
march_df = raw_data_by_month('03')
march_df = filter_data(march_df)
march_df = append_geocode_to_df(march_df)  

march_df.head()

/Users/znurtokatli/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,user_location,id,Latitude,Longitude
0,"3rd rock, NA, USA, NC",2,35.227087,-80.843127
1,"806 W. Main Street; Durham, NC",1,36.072635,-79.791975
2,"AVL, NC",1,35.779590,-78.638179
3,"Alamance, NC",1,35.227087,-80.843127


In [167]:
april_df = raw_data_by_month('04')
april_df = filter_data(april_df)
april_df = append_geocode_to_df(april_df) 

len(april_df)

4

In [168]:
may_df = raw_data_by_month('05')
may_df = filter_data(may_df)
may_df = append_geocode_to_df(may_df) 

len(may_df)

/Users/znurtokatli/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


4

In [171]:
# map func
def draw_map(month_df):
    
    loc_df = month_df[["Latitude", "Longitude"]].astype(float)
    
    gmaps.configure(api_key = gkey)
    map_figure = gmaps.figure()

    layer = gmaps.heatmap_layer(loc_df, weights = month_df['id'], dissipating = False, 
                                max_intensity = month_df['id'].max(), point_radius = 0.5, opacity = 0.3)

    map_figure.add_layer(layer)
    
    return map_figure
    

In [172]:
draw_map(march_df) 

Figure(layout=FigureLayout(height='420px'))

In [173]:
draw_map(april_df) 

Figure(layout=FigureLayout(height='420px'))

In [174]:
draw_map(may_df) 

Figure(layout=FigureLayout(height='420px'))